In [1]:
!pip install pymongo
!pip install langchain_community

In [2]:
from pymongo import MongoClient
from pymongo.database import Database
from pymongo.collection import Collection
import requests
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
import os
import pandas as pd
import numpy as np
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
# 토크나이저와 모델 로드
# embedding_tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
# embedding_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")


embedding_tokenizer =XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
embedding_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = embedding_model.to(device)

In [5]:
import os


MONGO_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')

DB_NAME = "test"

COLLECTION_NAME = "col1"

In [6]:
# MongoDB 클라이언트 생성


client = MongoClient(MONGO_URI)
collection = client[DB_NAME][COLLECTION_NAME]
# db: Database = client[DATABASE_NAME]
# collection: Collection = db[COLLECTION_NAME]

In [8]:
import pandas as pd

# csv 파일
input_file = "../용어사전_샘플링.csv"
df = pd.read_csv(input_file, dtype=str)

# print(df.head())

In [9]:
# 하이브리드 검색 가중치 설정
BM25_MAX_VALUE = 10.0  # 설정 필요
BM25_MIN_VALUE = 0.0   # 설정 필요
VECTOR_SCORE_WEIGHT = 0.5
TEXT_SCORE_WEIGHT = 0.5

In [11]:
# @title
def vector_search(query_vector, vector_index_name, num_candidates=64, limit=25):
    """
    벡터 검색 수행
    """
    pipeline = [
        {
            "$vectorSearch": {
                "index": vector_index_name,
                "path": "embedding",
                "queryVector": query_vector,
                "numCandidates": num_candidates,
                "limit": limit
            }
        },
        {
            "$project": {
                "metadata": 1,
                "content": 1,
                # "media": 1,
                "vectorScore": {"$meta": "vectorSearchScore"},
                "score": {"$meta": "vectorSearchScore"}
            }
        },
        {
            "$sort": {"score": -1}
        },
        {
            "$limit": limit
        }
    ]

    results = collection.aggregate(pipeline)
    return list(results)

In [12]:
# @title
def text_search(query, text_index_name, limit=25):
    """
    텍스트 검색 수행
    """
    pipeline = [
        {
            "$search": {
                "index": text_index_name,
                "text": {
                    "query": query,
                    "path": ["content", "metadata.KO", "metadata.ENG"]
                }
            }
        },
        {
            "$project": {
                "metadata": 1,
                "content": 1,
                # "media": 1,
                "textScore": {"$meta": "searchScore"},
                "score": {"$meta": "searchScore"}
            }
        },
        {
            "$sort": {"score": -1}
        },
        {
            "$limit": limit
        }
    ]

    results = collection.aggregate(pipeline)
    return list(results)

In [13]:
# @title
def get_embedding_from_xlm_roberta(text, model=embedding_model, tokenizer=embedding_tokenizer):
    """
    XLM-RoBERTa 모델을 사용해 텍스트 임베딩 생성

    Args:
        text (str or List[str]): 임베딩을 생성할 텍스트 또는 텍스트 리스트
        model: 사전 학습된 XLM-RoBERTa 모델
        tokenizer: 사전 학습된 XLM-RoBERTa 토크나이저

    Returns:
        List[float] or List[List[float]]: 입력 텍스트의 임베딩
    """

    # 입력 텍스트가 문자열인 경우 리스트로 변환
    if isinstance(text, str):
        text = [text]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # if model is not uploaded on device
    if not model.device.type == device:
        model = model.to(device)

    # 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델로 임베딩 생성
    with torch.no_grad():
        outputs = model(**inputs)

    # CLS 토큰 임베딩 사용
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().tolist()

    # 입력 텍스트가 단일 문장이었다면 첫 번째 임베딩만 반환
    if len(embeddings) == 1:
        return embeddings[0]
    return embeddings

In [14]:
# @title
def normalize_vector_score(vector_score):
    return (vector_score + 1) / 2.0

def normalize_bm25_score(bm25_score):
    return min((bm25_score - BM25_MIN_VALUE) / (BM25_MAX_VALUE - BM25_MIN_VALUE), 1.0)

def calculate_convex_score(vector_score, bm25_score):
    tmm_vector_score = normalize_vector_score(vector_score)
    tmm_bm25_score = normalize_bm25_score(bm25_score)
    return VECTOR_SCORE_WEIGHT * tmm_vector_score + TEXT_SCORE_WEIGHT * tmm_bm25_score

In [15]:
# @title
def hybrid_search(query, length=10, model=embedding_model, tokenizer=embedding_tokenizer):
    # 벡터 및 텍스트 검색 수행
    # embedding = get_embedding_from_azure(query)
    embedding = get_embedding_from_xlm_roberta(query, model, tokenizer)
    vector_results = vector_search(embedding, "word_vector_index")
    text_results = text_search(query,"word_text_index")

    # 결과 병합
    combined_results = {}
    for result in vector_results:
        doc_id = result["_id"]
        vector_score = result.get("vectorScore", 0)
        combined_results[doc_id] = {
            **result,
            "vectorScore": vector_score,
            "score": calculate_convex_score(vector_score, 0)
        }

    for result in text_results:
        doc_id = result["_id"]
        text_score = result.get("textScore", 0)
        if doc_id not in combined_results:
            combined_results[doc_id] = {
                **result,
                "vectorScore": 0,
                "score": calculate_convex_score(0, text_score)
            }
        else:
            vector_score = combined_results[doc_id]["vectorScore"]
            combined_results[doc_id]["textScore"] = text_score
            combined_results[doc_id]["score"] = calculate_convex_score(vector_score, text_score)

    # 결과 정렬
    sorted_results = sorted(combined_results.values(), key=lambda x: x["score"], reverse=True)
    return sorted_results[0:length]

In [16]:
# @title
def create_metadata_array(query, limit=10):
    """
    Hybrid Search를 수행하여 metadata를 array 형태로 묶은 JSON string 반환
    """
    # Hybrid Search 수행
    search_results = hybrid_search(query, limit)

    # metadata array 생성
    metadata_array = []
    for result in search_results[:limit]:
        metadata = result.get("metadata", {})
        # metadata의 key-value를 배열 형태로 변환
        # metadata_entry = [{"key": key, "value": value} for key, value in metadata.items()]

        # 배열 추가
        metadata_array.append(metadata)

    # metadata array를 JSON string으로 변환
    metadata_json = json.dumps(metadata_array, ensure_ascii=False)

    return metadata_json

In [ ]:
import uuid
# MongoDB에 저장할 데이터 생성 및 삽입
for _, row in df.iterrows():
    _content = '\t'.join(map(str, row.tolist()))
    _embedding = get_embedding_from_xlm_roberta(_content, embedding_model, embedding_tokenizer)
    document = {
        "_id": str(uuid.uuid4()),
        "metadata": {col: row[col] for col in df.columns},  # 각 컬럼을 metadata에 저장
        "content": '\t'.join(map(str, row.tolist())),  # content에 탭으로 연결된 값
        "media": [],
        "embedding": _embedding,  # Embedding 데이터가 없으므로 빈 리스트로 유지
        "contentFormatter": {
            "metadataTemplate": "{key}: {value}",
            "metadataSeparator": "\r\n",
            "textTemplate": "{metadata_string}\n\n{content}",
            "excludedInferenceMetadataKeys": [],
            "excludedEmbedMetadataKeys": [],
            "_class": "org.springframework.ai.document.DefaultContentFormatter"
        },
        "_class": "org.springframework.ai.document.Document"
    }
    collection.insert_one(document)

print("데이터가 성공적으로 MongoDB에 저장되었습니다!")

데이터가 성공적으로 MongoDB에 저장되었습니다!


In [17]:
query = "어딜 놀러갈까? 꾸러기어린이도서관은 어때? 관덕정 순교기념관은 어때? 나는 킨텍스에 관심이 있어."
type_filter = "report"
hybrid_search(query)

[{'_id': '9495abf6-8765-43fa-8bbb-0a8e63a36ee1',
  'metadata': {'KO': '늘푸른어린이도서관',
   'ENG': "Neulpureun Children's Library",
   'JPN': 'いつも青い子供図書館'},
  'content': "늘푸른어린이도서관\tNeulpureun Children's Library\tいつも青い子供図書館",
  'vectorScore': 0.9995787143707275,
  'score': 0.9953658223152161,
  'textScore': 9.909422874450684},
 {'_id': 'dd28d5a4-595c-4d4d-bd19-f984c39c15f0',
  'metadata': {'KO': '대구북구어린이도서관',
   'ENG': "Daegu Buk-gu Children's Library",
   'JPN': '大邱北区子供図書館'},
  'content': "대구북구어린이도서관\tDaegu Buk-gu Children's Library\t大邱北区子供図書館",
  'vectorScore': 0.9995675086975098,
  'score': 0.9721206903457642,
  'textScore': 9.444576263427734},
 {'_id': '274bfb41-df36-4063-9186-9835ba4d902c',
  'metadata': {'KO': '성남시중원어린이도서관',
   'ENG': "Seongnam City Jungwon Children's Library",
   'JPN': '城南市中院子供図書館'},
  'content': "성남시중원어린이도서관\tSeongnam City Jungwon Children's Library\t城南市中院子供図書館",
  'vectorScore': 0.9995789527893066,
  'score': 0.9509982347488404,
  'textScore': 9.022069931030273},
 

In [18]:
create_metadata_array(query, 10)

'[{"KO": "늘푸른어린이도서관", "ENG": "Neulpureun Children\'s Library", "JPN": "いつも青い子供図書館"}, {"KO": "대구북구어린이도서관", "ENG": "Daegu Buk-gu Children\'s Library", "JPN": "大邱北区子供図書館"}, {"KO": "성남시중원어린이도서관", "ENG": "Seongnam City Jungwon Children\'s Library", "JPN": "城南市中院子供図書館"}, {"KO": "관덕정 순교기념관", "ENG": "Kwandeokjeong Martyrs\' Memorial Hall", "JPN": "観徳亭殉教記念館"}, {"KO": "꾸러기어린이도서관", "ENG": "Kkureogi Children\'s Library", "JPN": "クロギ子供図書館"}, {"KO": "킨텍스", "ENG": "KINTEX", "JPN": "キンテックス"}, {"KO": "단원어린이도서관", "ENG": "Danwon Children\'s Library", "JPN": "檀園子供図書館"}, {"KO": "서구어린이도서관", "ENG": "Seo-gu Children\'s Library", "JPN": "西区子供図書館"}, {"KO": "개똥벌레어린이도서관", "ENG": "Gaettongbeolle Children\'s Library", "JPN": "蛍子供図書館"}, {"KO": "민들레어린이도서관", "ENG": "Mindeulle Children\'s Library", "JPN": "ミンドュレ子供図書館"}]'

In [19]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [20]:
!pip install huggingface_hub


In [24]:
from huggingface_hub import login


HF_TOKEN = os.getenv('HUGGING_FACE_TOKEN')


login(token=HF_TOKEN)



In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

# model, tokenizer = AutoTokenizer.from_pretrained(
#     model_name="meta-llama/Meta-Llama-3-8B-Instruct", # YOUR MODEL YOU USED FOR TRAINING
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
# AutoTokenizer.for_inference(model) # Enable native 2x faster inference

: 

In [ ]:
prompt_template_str = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Translate the original text into the target language
Adhere to the glossary for applicable terms

If a term does not have an entry in the target language,
adapt the most relevant one or translate appropriately based on context.

Apply proper capitalization rules for general nouns,
ensuring they are lowercase in the middle of a sentence unless they are proper nouns
or explicitly marked as capitalized in the glossary.

For terms like Scourge,
use lowercase and pluralize if the context suggests multiple entities.

Provide only the translated text without explanations.

### target language ###
{target_language}

### glossary ###
{glossary}

<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_message}

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

# PromptTemplate 정의
prompt_template = PromptTemplate(
    input_variables=["target_language", "glossary", "user_message"],  # 사용자가 입력할 변수
    template=prompt_template_str,
)

In [ ]:
# Hugging Face 파이프라인 설정
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #max_length=128,        # 생성될 텍스트의 최대 길이
    #temperature=0.7,       # 출력 다양성 조정
    #top_k=50,              # Top-k 샘플링
    #top_p=0.9,             # Nucleus 샘플링
    return_full_text=False,
)

Device set to use cuda:0


In [ ]:
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

# LangChain에서 Hugging Face 파이프라인 Wrapping
llm = HuggingFacePipeline(pipeline=hf_pipeline)


# LLMChain 설정
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

<ipython-input-25-bb63e016e597>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
user_message = """오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.
"""

target_language = "ENG"

In [ ]:
chain.run({
    "target_language": target_language,
    "glossary": create_metadata_array(user_message, 10),
    "user_message": user_message
}).lstrip("\n")

'님이신고\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심을 먹을거야. 저녁엔 킨텍스에 가자.\n\n\narsimp\n오늘 일정은 꾸러기어린이도서관을 가서 책을 보고, 관덕정 순교기념관 근처에서 점심

In [ ]:
from langchain.chat_models import ChatOpenAI
import openai

os.getenv('OPENAI_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

# OpenAI 모델 초기화
llm_azure = ChatOpenAI(model="gpt-4o-mini",
                       temperature=0,
                       openai_api_key = openai_api_key)


<ipython-input-37-68061d18a7c2>:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_azure = ChatOpenAI(model="gpt-4o-mini",


In [ ]:
prompt_template_str2 = """
### Role ###
This is a translation task for text which is related for traveling to Korea.
Evaluate the quality of the translation by reviewing the original text and its translation below Evaluation.
If necessary, suggest alternative translations below Suggestion.
Do not set the standards too high; approve the translation as long as the overall meaning is similar.
Refer to a dictionary if needed.
You must Explain in Korean.
Answer by the format given below.

### Original Text ###
{original_text}

### Target Language ###
{target_language}

### Glossary ###
{glossary}

### Translated Text ###
{translated_text}

### Evaluation ###

### Suggestion ###
"""

# PromptTemplate 정의
prompt_template2 = PromptTemplate(
    input_variables=["original_text", "target_language" "glossary", "translated_text"],  # 사용자가 입력할 변수
    template=prompt_template_str2,
)

In [ ]:
# LLMChain 설정
chain_azure = LLMChain(llm=llm_azure, prompt=prompt_template2)

In [ ]:
translated_text = chain.run({
    "target_language": "ENG",
    "glossary": create_metadata_array(user_message, 10),
    "user_message": user_message
}).lstrip("\n")

In [ ]:
explain = chain_azure.run({
    "target_language": "ENG",
    "glossary": create_metadata_array(user_message, 10),
    "original_text": user_message,
    "translated_text": translated_text
})

print(explain)